# Intro to SQL

## Bigquery

Build SQL skills using BigQuery (a web service to use SQL on huge datasets).  
First, basics about BigQuery datasets:

In [1]:
from google.cloud import bigquery

In [4]:
# This cell is to set the enviromental variable for the google credential
import os

path_to_credential = "C:\\Users\\fdoli\\Downloads\\UD01-9cd33c07ad1d.json"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_credential

In [5]:
# Create a "Client" object
client = bigquery.Client()

We'll use a dataset of posts on *Hacker News*.  

In BigQuery, each dataset is contained in a corresponding project. In this case, 
the `hacker_news` dataset is contained in the `bigquery-public-data` project.  
To access the dataset:
- We begin constructing a reference with the `dataset()` method. 
- We use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.  

In [7]:
# Construct a reference to the dataset
dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [8]:
# List all tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are 4!)
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


Similarly we can fetch a table.  
We fetch the `full` table in the `hacker_news` dataset

In [19]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table('full')

# API request - fetch the table
table = client.get_table(table_ref)

The flow diagram of what we have done until now is:

`client \ bigquery-publi-data \ hacker_news \ (comments, full, full_201510, stories)`

### Table schema

The structure of the table is called its schema. It's important to understand it:

Let's investigate the `full` table we fetched already:

In [20]:
# Print info on all the collumns in the `full` table in the `hacker_news` dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ()),
 SchemaField(

The information in the columns is organized as:
- The **name** of the column
- The **field type** in the column
- The **mode** of the column (`NULLABLE` means that a column allows NULL values, and is the default)
- A **description** of the data in that column

To check the rows information we use the `list_rows()` method. It shows the first 5 rows. The returned object (`RowIterator`) can be converted to a pandas DataFrame with the `to_dataframe()` method.

In [23]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"Right. To avoid confusion, I updated the post ...",None,BrendanEich,NaN,1334519257,2012-04-15 19:47:37+00:00,comment,3844505,3844394,NaN,NaN,None
1,None,None,It is possible to use only one or the other.,None,arkitectual,NaN,1482988141,2016-12-29 05:09:01+00:00,comment,13275737,13275001,NaN,NaN,None
2,None,None,"Educreations (<a href=""http://www.educreations...",None,streeter,NaN,1338572700,2012-06-01 17:45:00+00:00,comment,4054641,4053076,NaN,NaN,None
3,None,None,<i>Everyone</i> thinks in terms of pixel count...,None,mcguire,NaN,1416590679,2014-11-21 17:24:39+00:00,comment,8642235,8641060,NaN,NaN,None
4,None,None,&gt; Spamnesty is a way to waste spammers&#x27...,None,defanor,NaN,1514737026,2017-12-31 16:17:06+00:00,comment,16041080,16036121,NaN,NaN,None


In [28]:
# Preview the first five entries in the first colum of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


In [31]:
len(tables)

4

## Select, From & Where

Let's write the first SQL query! The keywords **SELECT**, **FROM**, and **WHERE** will help us.

### SELECT ... FROM  
this basic SQL query selects a *single* column from a single table. We should:
- specify the column we want after the word **SELECT**, and then
- specify the table after the word **FROM**

`query = """
        SELECT Name
        FROM 'bigquery-public-data.pet_records.pets'
        """`  
This will select the `Name` column from the `pets` table in the `pet_records` database in the `bigquery-public-data` project.

### WHERE ...  
This is to select specific rows from a column, to avoid returning large amounts of data from the large datasets.  

`query = """ 
         SELECT Name
         FROM 'bigquery-public-data.pet_records.pets'
         WHERE Animal = 'Cat'
         """`  
This query will return the entries from the `Name` column that are in rows where the `Animal` column has the text `Cat'.

### Example: What are all the U.S. cities in the OpenAQ dataset?  
Let's work with a real dataset. We'll use an OpenAQ dataset about air quality. 

In [32]:
from google.cloud import bigquery

# Create a 'Client' object
client = bigquery.Client()

# Construct a reference to the 'openaq' dataset
dataset_ref = client.dataset('openaq', project='bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the 'openaq' dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset 
for table in tables:
    print(table.table_id)

global_air_quality


Let's fetch the table and check its content

In [33]:
# Construct a reference to the 'global_air_quality' table
table_ref = dataset_ref.table('global_air_quality')

# API request - fetch the table!
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [34]:
table.schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', 'Longitude in d

Let's query it!

In [39]:
# Query to select all the items from the 'city' column where the 'country' is 'US'
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

### Submitting the query to the dataset  
We're ready to use this query to get the info from the OpenAQ dataset. We start by creating a `Client` object.

In [40]:
client = bigquery.Client()

In [41]:
# Set up the query
query_job = client.query(query)

Now, we run the query and convert the results to a pandas DataFrame.

In [42]:
# API request - run the query and return a pandas DataFrame
us_cities = query_job.to_dataframe()

Now we can work with the pandas dataframe

In [43]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale             88
Houston                             82
Los Angeles-Long Beach-Santa Ana    71
Riverside-San Bernardino-Ontario    60
San Francisco-Oakland-Fremont       58
Name: city, dtype: int64

### More queries  
If one wants multiple columns, you can select them with a comma between the names:  
`query = """
         SELECT city, country
         FROM 'bigquery-public-data.openaq.global_air_quality' 
         WHERE country = 'US'
         """`
         
One can select all the columns with a `*`: 

`query = """
         SELECT *
         FROM 'bigquery-public-data.openaq.global_air_quality' 
         WHERE country = 'US'
         """`
         

You can estimate the size of *any* query before running it. For that, we create a `QueryJobConfig` object and set the `dry_run` parameter to `True`

In [67]:
# Query to get two columns: score and title. Only data where the *type* column has value 'job'
query = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = 'job'
        """

# Create a QueryJobConfig object to estimate size of query **without running it**
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print('This query will process {} mega bytes.'.format(dry_run_query_job.total_bytes_processed/1000/1000))

This query will process 391.57072700000003 mega bytes.


In [52]:
# Lest check this one I already ran 
query2 = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
dry_run_query_job2 = client.query(query2, job_config=dry_run_config)
print('This query will process {} mega bytes.'.format(dry_run_query_job2.total_bytes_processed/1000/1000))

This query will process 0.415636 mega bytes.


Almost half a MB!

Following the same direction, you can set a limit to how much data you are willing to scan.  
For example:

In [72]:
query4 = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = 'job'
        """

# Only run the query if it is less than 1 MB
ONE_MB = 10
safe_config2 = bigquery.QueryJobConfig(maximum_bytes_billed=10)

# Set up the query (will only run if it is less than 1 MB)
safe_query_job = client.query(query4, job_config=safe_config2)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

In [70]:
job_post_scores.score.mean()

1.9125741595253791

In [73]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [74]:
from google.cloud import bigquery

In [75]:
# Create a "Client" object
client = bigquery.Client()

In [79]:
# Query to get two columns: score and title. Only data where the *type* column has value 'job'
query = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = 'job'
        """

# Create a QueryJobConfig object to estimate size of query **without running it**
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print('This query will process {} mega bytes.'.format(dry_run_query_job.total_bytes_processed/1000/1000))

This query will process 391.57072700000003 mega bytes.


In [92]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [93]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

In [94]:
query4 = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = 'job'
        """

# Only run the query if it is less than 1 MB
ONE_MB = 10
safe_config2 = bigquery.job.QueryJobConfig(maximum_bytes_billed=10)

# Set up the query (will only run if it is less than 1 MB)
safe_query_job = client.query(query4, job_config=safe_config2)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

In [95]:
job_post_scores.score.mean()

1.9125741595253791

---
## Group By, Having & Count

With these new commands we'll learn how to group the data and count things within those groups.

### COUNT()  
It returns a count of things. IF you pass the name of a column, it'll return the number of entries in that column. 

`query = """
        SELECT COUNT(ID)
        FROM 'bigquery-public-data.pet_records.pets'
        """`

**COUNT()** is an example of an aggregate function: it takes many values and returns one. (Other agg. functions are **SUM()**, **AVG()**, **MIN()** and **MAX()**.) These functions introduce strange columns names, but we can change that. 

### GROUP BY  
It takes the all the rows with the same value in a column as a single group when you apply agg. functions like **COUNT()**

`query = """
        SELECT Animal, COUNT(ID)
        FROM 'bigquery-public-data.pet_records.pets'
        GROUP BY Animal
        """`
        
### GROUP BY ... HAVING  
**HAVING** is used in combination with **GROUP BY** to ignore groups don't meet certain criteria.  
This query will include groups that have more than one ID in them.

`query = """
        SELECT COUNT(ID)
        FROM 'bigquery-public-data.pet_records.pets'
        GROUP BY Animal
        HAVING COUNT(ID)>1
        """`

Example!

In [96]:
%reset
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')

# API request
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the 'comments' table
table_ref = dataset_ref.table('comments')

# API request
table = client.get_table(table_ref)

# Preview the first five items of the 'comments' table
client.list_rows(table, max_results=5).to_dataframe()

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


We can find the comment that generated the mos replies, since:
- the `parent` column indicates the comment that was replied to, and 
- the `id` column has the unique ID used to identify each comment, 

we can **GROUP BY** the `parent` column and **COUNT()** the `id` column to se the number of responses to that comment. 

We can even set a lower limit to the **HAVING**

In [102]:
query_popular = """
                SELECT parent, COUNT(id) AS Replies
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

In [105]:
# Set up the query limit
safe_config3 = bigquery.QueryJobConfig(maximum_bytes_billed=1)
query_job = client.query(query_popular, job_config=safe_config3)

# API request 
popular_comments = query_job.to_dataframe()

# Print the first five rows of the data frame
popular_comments.head()

,parent,Replies
0,4684384,87
1,6584683,47
2,9616946,78
3,7750036,57
4,8185461,63


### Aliasing and other improvements

- The column resulting from `COUNT(id)` was called `f0_`. You can change that by adding `AS "NEWNAME"` after specifying the aggregation. This is called **aliasing**. 
- You can do `COUNT(1)` to count the rows in each group. Most people find it specially readable, because we know it's not focusing on other columns.  It also scans less data than if supplied column names (making it faster and using less of your data access quota).

---
## Order By

It is usually the last clause in a query, and it sorts the results returned by the rest of your query. 

`query = """
        SELECT ID, Name, Animal
        FROM 'bigquery-public-data.pet_records.pets'
        ORDER BY ID 
        """`

If you *order* a column that contains text, the results show up in alphabetical order. 

You can rever the *order* by using the **DESC** argument (*descending*). 

`query = """
        SELECT ID, Name, Animal
        FROM 'bigquery-public-data.pet_records.pets'
        ORDER BY Animal DESC
        """`
        
### Dates

Dates are very common in databases: They can be stores as **DATE** or as **DATETIME**. The **DATE** format has the year first, then the month, and then the day:
`YYYY - [M]M - [D]D`. You can have either one or two digits for monrth and day.  

The **DATETIME** format is like the date format, but with the time added at the end.


## EXTRACT  

You can use **EXTRACT** to obtain just a part of a date, such as only the year, or the day.  
`query = """
        SELECT Name, EXTRACT(DAY from Date) AS Day
        FROM 'bigquery-public-data.pet_records.pets_with_date'
        """`
        
You can even extract the week in the year (between 1 and 53) for each date in the `Date` column.  
`query = """                                                  
        SELECT Name, EXTRACT(WEEK from Date) AS Week          
        FROM 'bigquery-public-data.pet_records.pets_with_date'
        """`                                                  
        
More info in [here](https://cloud.google.com/bigquery/docs/reference/legacy-sql#datetimefunctions).

Example: Which day of the week has the most fatal motor accidents?

Using the *US Traffic Fatality Records* database, which contains information on traffic accidents in the US where at least one person died. 

Let's investigate the `accident_2015` table:

In [109]:
%reset
from google.cloud import bigquery

# Create 'client' object
client = bigquery.Client()

# Construct a reference to the 'nhtsa_traffic_fatalities' dataset
dataset_ref = client.dataset('nhtsa_traffic_fatalities', project='bigquery-public-data')

# API request- fetch it!
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the 'accident_2015' table
table_ref = dataset_ref.table('accident_2015')

# API request - fetch the table!
table = client.get_table(table_ref)

# Preview the first five lines of the 'accident_2015' table
client.list_rows(table, max_results=5).to_dataframe()

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00


We can use the table to see how the number of accidents varies with the day of the week. Since:
- the `consecutive_number` column contains a unique ID for each accident, and
- the `timestamp_of_crash` column contains the date of the accident in DATETIME format,  

we can:  
- **EXTRACT** the day of the week (as `day_of_the_week` in the query below) from the `timestamp_of_crash` column, and
- **GROUP BY** the day of the week, before we **COUNT** the `consecutive_number` column to determine the number of accidents for each day of the week. 

Then we can sort the table with an **ORDER BY** clause, so we put the days with more accidents on top: 

In [112]:
# Query to find the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

In [113]:
# Set up the query (cancel the query if it would use too much of the quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**6)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


From the [documentation](https://cloud.google.com/bigquery/docs/reference/legacy-sql#dayofweek), we can see that `1 = Sunday` and `7 = Saturday`. So, in 2015, most fatal motor accidents in the US ocurred on Sunday and Saturday, while the fewest happened on Tuesday.

## As & With

These are used to tidy up the queries and make them easier to read, because they can get very long.  
Noy we use as example a `pets` table with the `Name`, `Animal` and `Years_old` columns.

### AS
We already saw this is for renaming the columns generated by the queries.  
`query = """
        SELECT Animal, COUNT(ID) AS Number 
        FROM 'bigquery-public-data.pet_records.pets'
        GROUP BY Animal
        """`
        
### WITH ... AS  
**AS** can be combined with **WITH** in a 'common table expression' (**CTE**). This is a temporary table that one returns within the query. CTEs are helpful for splitting the queries into readable chunks, and one can write queries against them. 

`query = """
         WITH Seniors AS
         (
             SELECT ID, Name
             FROM 'bigquery-public-data.pet_records.pets'
             WHERE Years_old > 5
         )
         SELECT ID
         FROM Seniors
         """`
         
CTEs only exist ***inside the query** they are created and cannot be referenced in later queries. So the query using CTE can be broken in: (1) first, we create the CTE, and then (2) we write a query that uses the CTE. 


Example: How many Bitcoin transactions are made per month?  
We'll use a CTE for that. We'll investigate the `transactions` table. 

In [120]:
%reset
from google.cloud import bigquery

# Create 'client' object
client = bigquery.Client()

# Construct a reference to the 'crypto_bitcoin' dataset
dataset_ref = client.dataset('crypto_bitcoin', project='bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the 'transactions' table
table_ref = dataset_ref.table('transactions')

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the 'transactions' table
###---this next line gives an error in the conversion to pandas DataFrame---### 
#client.list_rows(table, max_results=5).to_dataframe()

Once deleted, variables cannot be recovered. Proceed (y/[n])?  n


Nothing done.


In **conclusion**, CTEs help cleaning the code and increasing efficiency when selecting data as it is faster than doing it in Pandas. 

### Sample query from exercises: 
    
`speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, trip_seconds
                   FROM 'bigquery-public-data.chicago_taxi_trips.taxi_trips'
                   WHERE trip_start_timestamp > '2017-01-01' AND
                         trip_start_timestamp < '2017-07-01' AND
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600*SUM(trip_miles)/SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """
` 

---
## Joining Data

Useful when the data needed is spread across multiple tables. It's very important for the SQL workflow. 

### JOIN  
It used when you want to join tables column-wise, even when they are not ordered in the same way, but you can match a column in common. This common column will do the sorting out. 

`query = """
         SELECT p.Name AS Pet_Name, o.Name AS Owner_name
         FROM 'bigquery-public-data.pet_records.pets' AS p
         INNER JOIN 'bigquery-public-data.pet_records.owners' AS o
             ON p.ID = o.Pet_ID
         """
         ` 
         
Here we combine information from both tables by matching rows where the `ID` column in the `pet` table matches the `Pet_ID` column in the `owners` table.  
In the query, **ON** determines which column in each table to use to combine the tables. Since they have the same name in both columns, we have to clarify which one to use. We use `p.ID` and `o.ID` for that.  

    In general, this is a good practice. To specify which table each of the columns come from.  
    
There are many other types of **JOIN**, this one, the **INNER JOIN**, will only show in the final table the values that appear in both tables that are been used. 

Here is another query that looks up the githu repositories to count how many files use the different types of software legal licence. 

`query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM 'bigquery-public-data.github_repos.sample_files' AS sf
        INNER JOIN 'bigquery-public-data.github_repos.licenses' AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """` 
        
Again, we do an **INNER JOIN** to get only the values present in both tables. And we join them at the `repo_name` column of both tables, specified in the `ON` instruction. 

---
## LIKE  
One can use **LIKE** instructions together with **WHERE** to restrict even more the results to rows with certain text.  

`query = """
        SELECT *
        FROM 'bigquery-public-data.pet_records.pets'
        WHERE Name LIKE 'Ripley'
        """` 
        
Similar to *Regular Expressions* characters, you can use `%` as a 'wildcard' for any number of characters, so the same as the previous query can be written as:

`query = """
        SELECT *
        FROM 'bigquery-public-data.pet_records.pets'
        WHERE Name LIKE '%ipl%'
        """`

